In [45]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt


# Importing the data from the data set

In [46]:
df = pd.read_csv(r'C:\Users\gdmarques\OneDrive - CRITICAL SOFTWARE, S.A\Documents\Customer Segmentation\customer-segmentation\customer_segmentation\data\bank_transactions.csv')
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR)
0,T1,C5841053,10/1/94,F,JAMSHEDPUR,17819.05,2/8/16,143207,25.0
1,T2,C2142763,4/4/57,M,JHAJJAR,2270.69,2/8/16,141858,27999.0
2,T3,C4417068,26/11/96,F,MUMBAI,17874.44,2/8/16,142712,459.0
3,T4,C5342380,14/9/73,F,MUMBAI,866503.21,2/8/16,142714,2060.0
4,T5,C9031234,24/3/88,F,NAVI MUMBAI,6714.43,2/8/16,181156,1762.5


# Data Cleanup

Dropping missing data

In [47]:
df.dropna(inplace = True)

Check for duplicated values

In [48]:
df.duplicated().sum()

np.int64(0)

Both the date of birth and transaction date are in weird types, so I will convert them to datetime format so it is easier to interpret

In [49]:
def convert_date(date_str):
    date = pd.to_datetime(date_str, format='%d/%m/%y', errors='coerce')
    # Adjust century for dates that are incorrectly in the 2000s
    if date.year > pd.Timestamp.now().year:
        date = date - pd.DateOffset(years=100)
    return date

df['TransactionDate'] = df['TransactionDate'].apply(convert_date)
df['CustomerDOB'] = df['CustomerDOB'].apply(convert_date)


I will create a new column called "CustomerAge" because the age of the customer when each transaction was made. 
For a bank it is relevant to have this information, because it provides insights into spending behavior, product needs, risk assessment, and customer engagement strategies. 
By analyzing this information, banks can enhance their offerings, improve customer experience, and foster long-term relationships with customers.

In [50]:
df['CustomerAge'] =df['TransactionDate'].dt.year - df['CustomerDOB'].dt.year

In [51]:
df.head()

,TransactionID,CustomerID,CustomerDOB,CustGender,CustLocation,CustAccountBalance,TransactionDate,TransactionTime,TransactionAmount (INR),CustomerAge
0,T1,C5841053,1994-01-10,F,JAMSHEDPUR,17819.05,2016-08-02,143207,25.0,22.0
1,T2,C2142763,1957-04-04,M,JHAJJAR,2270.69,2016-08-02,141858,27999.0,59.0
2,T3,C4417068,1996-11-26,F,MUMBAI,17874.44,2016-08-02,142712,459.0,20.0
3,T4,C5342380,1973-09-14,F,MUMBAI,866503.21,2016-08-02,142714,2060.0,43.0
4,T5,C9031234,1988-03-24,F,NAVI MUMBAI,6714.43,2016-08-02,181156,1762.5,28.0
